In [1]:
from pathlib import Path
import glob
import pandas as pd


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 1000)    


%load_ext autoreload
%autoreload 2

In [2]:
data_dir = Path('../data/raw/')
data_files = list(data_dir.glob('*.csv'))

data_dict = {
    file.stem.split('_')[0]: pd.read_csv(file) for file in data_files
}


In [3]:
df = data_dict['healthcare']
df = df.copy()
df.head()

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
0,Bobby JacksOn,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal
1,LesLie TErRy,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive
2,DaNnY sMitH,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal
3,andrEw waTtS,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,"Hernandez Rogers and Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal
4,adrIENNE bEll,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal


## Name standardisation 

In [4]:
# name standardization
df['Name'] = df['Name'].str.title().str.strip()
df['Name'][:10]

0         Bobby Jackson
1          Leslie Terry
2           Danny Smith
3          Andrew Watts
4         Adrienne Bell
5         Emily Johnson
6        Edward Edwards
7    Christina Martinez
8       Jasmine Aguilar
9      Christopher Berg
Name: Name, dtype: object

## Dropping Duplicates

In [5]:
# Dropping Duplicates
print(f"DataFrame shape before dropping duplicates: {df.shape}")
original_shape = df.shape
print(f"Number of duplicate rows: {df.duplicated().sum()}")
df = df.drop_duplicates(keep='first').reset_index(drop=True)
print(f"DataFrame shape after dropping duplicates: {(original_shape[0]-df.shape[0])/df.shape[0]*100:.2f}% reduction in rows.")

DataFrame shape before dropping duplicates: (55500, 15)
Number of duplicate rows: 534
DataFrame shape after dropping duplicates: 0.97% reduction in rows.


## DATA TYPE OPTIMIZATION

In [6]:
## Data type optimisation 
df['Date of Admission'] = pd.to_datetime(df['Date of Admission'])
df['Discharge Date'] = pd.to_datetime(df['Discharge Date'])
df['Hospital'] = df['Hospital'].astype('category')
df['Medical Condition'] = df['Medical Condition'].astype('category')
df['Billing Amount'] = pd.to_numeric(df['Billing Amount'], errors='coerce')
df.info(memory_usage='deep')
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54966 entries, 0 to 54965
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Name                54966 non-null  object        
 1   Age                 54966 non-null  int64         
 2   Gender              54966 non-null  object        
 3   Blood Type          54966 non-null  object        
 4   Medical Condition   54966 non-null  category      
 5   Date of Admission   54966 non-null  datetime64[ns]
 6   Doctor              54966 non-null  object        
 7   Hospital            54966 non-null  category      
 8   Insurance Provider  54966 non-null  object        
 9   Billing Amount      54966 non-null  float64       
 10  Room Number         54966 non-null  int64         
 11  Admission Type      54966 non-null  object        
 12  Discharge Date      54966 non-null  datetime64[ns]
 13  Medication          54966 non-null  object    

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
0,Bobby Jackson,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal
1,Leslie Terry,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive
2,Danny Smith,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal
3,Andrew Watts,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,"Hernandez Rogers and Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal
4,Adrienne Bell,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal


## Quality check 

In [7]:
# runining a quality check in a dataframe format 
quality_check = pd.DataFrame({
    'Data Type': df.dtypes,
    'Missing Values': df.isnull().sum(),
    'Missing %': round((df.isnull().sum() / len(df)) * 100, 2),
    'Unique Values': df.nunique(),
    'Duplicate Count': [df[col].duplicated().sum() for col in df.columns],
    'Duplicate %': [round((df[col].duplicated().sum() / len(df)) * 100, 2) for col in df.columns],
    'Cardinality': [
        'High' if df[col].nunique() / len(df) > 0.5 
        else 'Medium' if df[col].nunique() / len(df) > 0.05 
        else 'Low' 
        for col in df.columns
    ],
    'Most Common Value': [df[col].mode()[0] if len(df[col].mode()) > 0 else None for col in df.columns],
    'Most Common Freq': [df[col].value_counts().iloc[0] if len(df[col]) > 0 else 0 for col in df.columns],
    'Most Common %': [round((df[col].value_counts().iloc[0] / len(df)) * 100, 2) if len(df[col]) > 0 else 0 for col in df.columns],
})

quality_check

,Data Type,Missing Values,Missing %,Unique Values,Duplicate Count,Duplicate %,Cardinality,Most Common Value,Most Common Freq,Most Common %
Name,object,0,0.0,40235,14731,26.80,High,Michael Williams,24,0.04
Age,int64,0,0.0,77,54889,99.86,Low,38,890,1.62
Gender,object,0,0.0,2,54964,100.00,Low,Male,27496,50.02
Blood Type,object,0,0.0,8,54958,99.99,Low,A-,6898,12.55
Medical Condition,category,0,0.0,6,54960,99.99,Low,Arthritis,9218,16.77
Date of Admission,datetime64[ns],0,0.0,1827,53139,96.68,Low,2024-03-16 00:00:00,50,0.09
Doctor,object,0,0.0,40341,14625,26.61,High,Michael Smith,27,0.05
Hospital,category,0,0.0,39876,15090,27.45,High,LLC Smith,44,0.08
Insurance Provider,object,0,0.0,5,54961,99.99,Low,Cigna,11139,20.27
Billing Amount,float64,0,0.0,50000,4966,9.03,High,-1316.618581,2,0.00


## Saving the cleaned data toa file 

In [8]:
#save the cleaned dataframe
cleaned_data_dir = Path('../data/cleaned/')
cleaned_data_dir.mkdir(parents=True, exist_ok=True)
df.to_csv(cleaned_data_dir / 'healthcare_cleaned.csv', index=False)
